In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()

In [2]:
dev = qml.device("default.qubit", wires=2)

In [3]:
def get_angles(x):

    beta0 = 2 * np.arcsin(np.sqrt(x[1]) ** 2 / np.sqrt(x[0] ** 2 + x[1] ** 2 + 1e-12))
    beta1 = 2 * np.arcsin(np.sqrt(x[3]) ** 2 / np.sqrt(x[2] ** 2 + x[3] ** 2 + 1e-12))
    beta2 = 2 * np.arcsin(np.sqrt(x[2] ** 2 + x[3] ** 2) / np.sqrt(x[0] ** 2 + x[1] ** 2 + x[2] ** 2 + x[3] ** 2))

    return np.array([beta2, -beta1 / 2, beta1 / 2, -beta0 / 2, beta0 / 2])

In [4]:
def statepreparation(a):
    qml.RY(a[0], wires=0)

    qml.CNOT(wires=[0, 1])
    qml.RY(a[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[2], wires=1)

    qml.PauliX(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[3], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(a[4], wires=1)
    qml.PauliX(wires=0)

In [5]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [6]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [7]:
x = np.array([0.53896774, 0.79503606, 0.27826503, 0.0])
ang = get_angles(x)
print(ang)

@qml.qnode(dev)
def test(angles=None):

    statepreparation(angles)

    return qml.expval(qml.PauliZ(0))


test(angles=ang)

print("x               : ", x)
print("angles          : ", ang)
print("amplitude vector: ", np.real(dev._state))

[ 0.56397465 -0.          0.         -0.97504604  0.97504604]
x               :  [0.53896774 0.79503606 0.27826503 0.        ]
angles          :  [ 0.56397465 -0.          0.         -0.97504604  0.97504604]
amplitude vector:  [ 5.38967743e-01  7.95036065e-01  2.78265032e-01 -1.38777878e-17]


In [8]:
def layer(W):
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.CNOT(wires=[0, 1])

In [9]:
@qml.qnode(dev)
def circuit(weights, angles=None):
    statepreparation(angles)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))


def variational_classifier(var, angles=None):
    weights = var[0]
    bias = var[1]
    return circuit(weights, angles=angles) + bias


def cost(weights, features, labels):
    predictions = [variational_classifier(weights, angles=f) for f in features]
    return square_loss(labels, predictions)

In [10]:
X_tra = []
Y_ans = []
with open('data_banknote_authentication.txt', 'r') as f:
    x = f.readlines()
random.shuffle(x)
for i in range(len(x)):
    x[i] = x[i][:-1]
    x[i] = x[i].split(',')
    X_tra.append(x[i][:-1])
    Y_ans.append(x[i][-1:])
x
ans = []
for i in range(len(Y_ans)):
    if(Y_ans[i]==min(Y_ans)):
        Y_ans[i]=['0']
    ans.append(Y_ans[i][0])
for i in range(len(X_tra)):
    ans[i]=int(ans[i])
    for j in range(len(X_tra[i])):
        X_tra[i][j]=float(X_tra[i][j])

In [15]:
print(scaler.fit(X_tra))

MinMaxScaler(copy=True, feature_range=(0, 1))


In [16]:
X_tra = scaler.transform(X_tra)
#X_tra

In [17]:
features = np.array([get_angles(x) for x in X_tra])
features

array([[ 1.86020929, -1.1687676 ,  1.1687676 , -0.962637  ,  0.962637  ],
       [ 1.00534856, -1.37791872,  1.37791872, -0.77795601,  0.77795601],
       [ 1.3411122 , -1.35038655,  1.35038655, -1.02891548,  1.02891548],
       ...,
       [ 1.11722747, -1.42178944,  1.42178944, -0.91724243,  0.91724243],
       [ 1.0965786 , -1.20837942,  1.20837942, -0.93721681,  0.93721681],
       [ 0.62799823, -1.49470424,  1.49470424, -0.81028782,  0.81028782]])

In [18]:
Y=np.array(ans)
Y

array([1, 0, 1, ..., 1, 0, 0])

In [19]:
#np.random.seed(42)
num_data = len(Y)
num_train = int(0.50 * num_data)
index = np.random.permutation(range(num_data))
print(index)
feats_train = features[index[:num_train]]
Y_train = Y[index[:num_train]]
feats_val = features[index[num_train:]]
Y_val = Y[index[num_train:]]

# We need these later for plotting
#X_train = X[index[:num_train]]
#X_val = X[index[num_train:]]

[ 506   56 1025 ... 1348  950  996]


In [ ]:
num_qubits = 2
num_layers = 6
var_init = (0.01 * np.random.randn(num_layers, num_qubits, 3), 0.0)

opt = NesterovMomentumOptimizer(0.01)
batch_size = 5

var = var_init
for it in range(60):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = feats_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    var = opt.step(lambda v: cost(v, feats_train_batch, Y_train_batch), var)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(var, angles=f)) for f in feats_train]
    predictions_val = [np.sign(variational_classifier(var, angles=f)) for f in feats_val]

    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_val = accuracy(Y_val, predictions_val)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
        "".format(it + 1, cost(var, features, Y), acc_train, acc_val)
    )

Iter:     1 | Cost: 0.7471448 | Acc train: 0.1953353 | Acc validation: 0.1618076 
Iter:     2 | Cost: 0.7184223 | Acc train: 0.2186589 | Acc validation: 0.1851312 
Iter:     3 | Cost: 0.6901069 | Acc train: 0.2376093 | Acc validation: 0.2055394 
Iter:     4 | Cost: 0.6635402 | Acc train: 0.2492711 | Acc validation: 0.2215743 
Iter:     5 | Cost: 0.6192659 | Acc train: 0.2857143 | Acc validation: 0.2696793 
Iter:     6 | Cost: 0.5965948 | Acc train: 0.3309038 | Acc validation: 0.2973761 
Iter:     7 | Cost: 0.5841229 | Acc train: 0.3586006 | Acc validation: 0.3294461 
Iter:     8 | Cost: 0.5767584 | Acc train: 0.3833819 | Acc validation: 0.3600583 
Iter:     9 | Cost: 0.5707850 | Acc train: 0.4067055 | Acc validation: 0.3848397 
Iter:    10 | Cost: 0.5652284 | Acc train: 0.4300292 | Acc validation: 0.3950437 
Iter:    11 | Cost: 0.5603645 | Acc train: 0.4387755 | Acc validation: 0.4096210 
Iter:    12 | Cost: 0.5448996 | Acc train: 0.4431487 | Acc validation: 0.4183673 
Iter:    13 | Co